In [3]:
import pandas as pd
import numpy as np

In [4]:
def write_granule_list(input_list):
    df = pd.read_csv(input_list, header=None, names=['fn','hold_reason'])
    df['is_memory'] = df.apply(lambda x: 'exceeded request_memory' in x.hold_reason, axis=1)
    df['granule'] = df.apply(lambda x: x.fn[x.fn.find('ATL03_'):x.fn.find('.h5')+3], axis=1)
    def get_description(x):
        substr = x.fn[x.fn.find('job_')+4:x.fn.find('_ATL03_')]
        return substr[:substr.rfind('-')]
    df['description'] = df.apply(get_description, axis=1)
    def get_geojson(x):
        parms = x.description.split('_')
        parms[0] = 'ANT' if parms[0] == 'AIS' else 'GRE'
        del parms[1]
        return 'geojsons/simplified_' + '_'.join(parms) + '.geojson'
    df['geojson'] = df.apply(get_geojson, axis=1)
    df['geojson_full'] = df.apply(lambda x: x.geojson.replace('simplified_', ''), axis=1)
    df = df[['granule','geojson','description','geojson_full','fn','hold_reason','is_memory']]
    df.to_csv(input_list.replace('.csv', '_processed.csv'))
    
    df_mem = df[df.is_memory]
    df_nomem = df[~df.is_memory]
    
    df_mem = df_mem.drop(columns=['fn','hold_reason','is_memory'])
    df_nomem = df_nomem.drop(columns=['fn','hold_reason','is_memory'])
    
    fn_mem = input_list.replace('hold_lists/', 'granule_lists/').replace('final_', 'memory_')
    fn_nomem = input_list.replace('hold_lists/', 'granule_lists/').replace('final_', 'resubmit_')
    
    df_mem.to_csv(fn_mem, header=False, index=False)
    print('Wrote file %s. (%i jobs)' % (fn_mem, len(df_mem)))
    df_nomem.to_csv(fn_nomem, header=False, index=False)
    print('Wrote file %s. (%i jobs)' % (fn_nomem, len(df_nomem)))

    return fn_mem, fn_nomem

def write_submit_file(list_fn, sub_fn=None, mem_gb=16): 
    if not sub_fn:
        sub_fn = list_fn.replace('granule_lists/', 'HTCondor_submit/').replace('.csv', '.submit')
    
    f = open(sub_fn, "w")
    print('universe    = vanilla', file=f)
    print('+SingularityImage = "osdf:///ospool/ap21/data/fliphilipp/containers/icelake-container_v1.sif"', file=f)
    print('Requirements = HAS_SINGULARITY == True && OSG_HOST_KERNEL_VERSION >= 31000', file=f)
    print('executable  = run_py.sh', file=f)
    print('arguments = $(granule) $(polygon)', file=f)
    print('max_retries = 30', file=f)
    print('success_exit_code = 69', file=f)
    print('transfer_input_files = detect_lakes.py, icelakes/__init__.py, icelakes/utilities.py, icelakes/nsidc.py, icelakes/detection.py, misc/test1, misc/test2, $(polygon), $(polygon_full)', file=f)
    print('transfer_output_files = detection_out_data, detection_out_plot, detection_out_stat', file=f)
    print('should_transfer_files = YES', file=f)
    print('when_to_transfer_output = ON_EXIT', file=f)
    print('log           = logs/job_$(descriptor)-$(ClusterID)_$(granule)-$(ProcID).log', file=f)
    print('error         = errs/job_$(descriptor)-$(ClusterID)_$(granule)-$(ProcID).err', file=f)
    print('output        = outs/job_$(descriptor)-$(ClusterID)_$(granule)-$(ProcID).out', file=f)
    print('request_cpus    = 1', file=f)
    print('request_memory  = %iGB' % mem_gb, file=f)
    print('request_disk    = %iGB' % mem_gb, file=f)
    print('queue granule,polygon,descriptor,polygon_full from %s' % list_fn, file=f)
    f.close()
    
    print('Wrote file %s.\n' % sub_fn)

## Inital submit files

In [3]:
write_submit_file('granule_lists/GRE_2000_May2019_Jun2023.csv', mem_gb=16)
write_submit_file('granule_lists/ANT_1000_Dec2018_Mar2021.csv', mem_gb=16)
write_submit_file('granule_lists/ANT_1000_Dec2021_Mar2023.csv', mem_gb=16)

Wrote file HTCondor_submit/GRE_2000_May2019_Jun2023.submit.

Wrote file HTCondor_submit/ANT_1000_Dec2018_Mar2021.submit.

Wrote file HTCondor_submit/ANT_1000_Dec2021_Mar2023.submit.



In [4]:
write_submit_file('granule_lists/extra_shoulderseason_GRE_2000_ANT_1000.csv', mem_gb=16)

Wrote file HTCondor_submit/extra_shoulderseason_GRE_2000_ANT_1000.submit.



## TODO before next run: 
- make sure code runs properly locally
- then change sleep time to 1000 or something 

In [11]:
! ls hold_lists

final_ant-2123-1-138198.csv
final_ant-2123-1-138198_processed.csv
final_ant1821-1-138197.csv
final_ant1821-1-138197_processed.csv
final_extra-months-1-169920.csv
final_extra-months-1-169920_processed.csv
final_gre1823-1-138199.csv
final_gre1823-1-138199_processed.csv


In [6]:
input_list = 'hold_lists/final_ant1821-1-138197.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

Wrote file granule_lists/memory_ant1821-1-138197.csv. (91 jobs)
Wrote file granule_lists/resubmit_ant1821-1-138197.csv. (1763 jobs)


In [8]:
input_list = 'hold_lists/final_ant-2123-1-138198.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

Wrote file granule_lists/memory_ant-2123-1-138198.csv. (53 jobs)
Wrote file granule_lists/resubmit_ant-2123-1-138198.csv. (56 jobs)


In [9]:
input_list = 'hold_lists/final_gre1823-1-138199.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

Wrote file granule_lists/memory_gre1823-1-138199.csv. (133 jobs)
Wrote file granule_lists/resubmit_gre1823-1-138199.csv. (867 jobs)


In [10]:
input_list = 'hold_lists/final_extra-months-1-169920.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

Wrote file granule_lists/memory_extra-months-1-169920.csv. (40 jobs)
Wrote file granule_lists/resubmit_extra-months-1-169920.csv. (1294 jobs)


In [22]:
file_list_resub1 = [
    'granule_lists/resubmit_ant1821-1-138197.csv',
    'granule_lists/resubmit_ant-2123-1-138198.csv',
    'granule_lists/resubmit_gre1823-1-138199.csv',
    'granule_lists/resubmit_extra-months-1-169920.csv'
]
out_name = 'granule_lists/resubmit_all_2.csv'
dfs = []
for fn in file_list_resub1:
    dfs.append(pd.read_csv(fn,header=None))
dfb = pd.concat(dfs).reset_index(drop=True)
dfb.to_csv(out_name, header=False, index=False)

print(out_name)
print('Number of ganules over Greenland:', np.sum(dfb.apply(lambda x: 'GrIS' in x.loc[2], axis=1)))
print('Number of ganules over Antarctica:', np.sum(dfb.apply(lambda x: 'AIS' in x.loc[2], axis=1)))
print('Total number of granules:', len(dfb))
write_submit_file(out_name, mem_gb=16)

granule_lists/resubmit_all_2.csv
Number of ganules over Greenland: 1980
Number of ganules over Antarctica: 2000
Total number of granules: 3980
Wrote file HTCondor_submit/resubmit_all_2.submit.



In [23]:
file_list_resub1 = [
    'granule_lists/memory_ant1821-1-138197.csv',
    'granule_lists/memory_ant-2123-1-138198.csv',
    'granule_lists/memory_gre1823-1-138199.csv',
    'granule_lists/memory_extra-months-1-169920.csv'
]
out_name = 'granule_lists/memory_resubmit_all_2.csv'
dfs = []
for fn in file_list_resub1:
    dfs.append(pd.read_csv(fn,header=None))
dfb = pd.concat(dfs).reset_index(drop=True)
dfb.to_csv(out_name, header=False, index=False)

print(out_name)
print('Number of ganules over Greenland:', np.sum(dfb.apply(lambda x: 'GrIS' in x.loc[2], axis=1)))
print('Number of ganules over Antarctica:', np.sum(dfb.apply(lambda x: 'AIS' in x.loc[2], axis=1)))
print('Total number of granules:', len(dfb))
write_submit_file(out_name, mem_gb=32)

granule_lists/memory_resubmit_all_2.csv
Number of ganules over Greenland: 135
Number of ganules over Antarctica: 182
Total number of granules: 317
Wrote file HTCondor_submit/memory_resubmit_all_2.submit.



In [ ]:
input_list = 'hold_lists/final_greenland-1-114211.csv'
fn_mem, fn_nomem = write_granule_list(input_list)
write_submit_file(fn_nomem)

In [ ]:
input_list = 'hold_lists/final_antarctica-18-21-1-114218.csv'
fn_mem, fn_nomem = write_granule_list(input_list)
write_submit_file(fn_nomem)

In [ ]:
input_list = 'hold_lists/final_antarctica-21-23-1-114219.csv'
fn_mem, fn_nomem = write_granule_list(input_list)
write_submit_file(fn_nomem)

In [ ]:
input_list = 'hold_lists/final_greenland-2-117771.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

In [ ]:
input_list = 'hold_lists/final_antarctica-18-21-2-123983.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

In [ ]:
input_list = 'hold_lists/final_antarctica-21-23-2-123993.csv'
fn_mem, fn_nomem = write_granule_list(input_list)

In [ ]:
outname = 'granule_lists/resubmit_all-3.csv'
filelist_2nds = [
    'granule_lists/resubmit_greenland-2-117771.csv',
    'granule_lists/resubmit_antarctica-18-21-2-123983.csv',
    'granule_lists/resubmit_antarctica-21-23-2-123993.csv'
]
dfs = []
for fn in filelist_2nds:
    df = pd.read_csv(fn, header=None)
    print(len(df), 'jobs', fn)
    dfs.append(df)
df_all = pd.concat(dfs)
df_all.to_csv(outname, header=False, index=False)
print('Wrote file %s. (%i jobs)' % (outname, len(df_all)))
write_submit_file(outname, mem_gb=16)

In [ ]:
input_list = 'hold_lists/final_resubmit_all-4-135549.csv'
fn_mem, fn_nomem = write_granule_list(input_list)
write_submit_file(fn_nomem)

In [ ]:
outname = 'granule_lists/memory_resubmit_all-4.csv'
filelist_mem = [
    'granule_lists/memory_antarctica-18-21-1-114218.csv',
    'granule_lists/memory_antarctica-18-21-2-123983.csv',
    # 'granule_lists/memory_antarctica-21-23-1-114219.csv', # no jobs
    'granule_lists/memory_antarctica-21-23-2-123993.csv',
    'granule_lists/memory_greenland-1-114211.csv',
    'granule_lists/memory_greenland-2-117771.csv',
    'granule_lists/memory_resubmit_all-4-135549.csv'
]
dfs = []
for fn in filelist_mem:
    df = pd.read_csv(fn, header=None)
    print(len(df), 'jobs', fn)
    dfs.append(df)
df_all = pd.concat(dfs)
df_all.to_csv(outname, header=False, index=False)
print('Wrote file %s. (%i jobs)' % (outname, len(df_all)))
write_submit_file(outname)